<a href="https://colab.research.google.com/github/RyuichiSaito1/multilingual-economic-narratives/blob/main/notebooks/classification_test_using_gpt_4_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install openai pandas gspread google-auth

print("Installation completed")

Mounted at /content/drive
Installation completed


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import re
import os
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

from google.colab import auth
from openai import OpenAI
import gspread
from google.auth import default

print("All libraries imported successfully")

All libraries imported successfully


In [ ]:
# For Minimal reasoning effort

# !pip uninstall -y openai
# !pip install --no-cache-dir --force-reinstall "openai==1.99.2"

In [3]:
print("Setting up OpenAI API...")

OPENAI_API_KEY = "OPENAI_API_KEY"

client = OpenAI(api_key=OPENAI_API_KEY)
print("OpenAI client initialized successfully!")

Setting up OpenAI API...
OpenAI client initialized successfully!


In [16]:
print("Testing GPT-4.1 mini model connection...")

try:
    test_response = client.chat.completions.create(
        model="gpt-4.1-mini",
        # Default: medium
        # reasoning={"effort": "low"},
        messages=[{"role": "user", "content": "Hello, respond with 'OK'"}],
        max_completion_tokens=10,
        temperature=0,
    )
    print(f"Model test response: {test_response.choices[0].message.content}")
    print("Successfully connected to GPT-4.1 mini")
except Exception as e:
    print(f"Error connecting to GPT-4.1 mini: {e}")
    print("Note: If GPT-4.1 mini is not available, you may need to check the model name or your API access")
    raise

Testing GPT-4.1 mini model connection...
Model test response: OK
Successfully connected to GPT-4.1 mini


In [37]:
class RedditEconomicSentimentClassifier:
    def __init__(self, client):

        self.client = client

        self.prompt_template = """You are the chief economist at the ESM. Classify the Reddit post below on TWO dimensions.

TASK 1 - SENTIMENT:
Choose 0 (positive), 1 (neutral), or 2 (negative)

0: indicates positive, such as admire, amazing, assure, celebration, charm, eager, enthusiastic, excellent, fancy, fantastic, frolic, graceful, happy, joy, luck, majesty, mercy, nice, patience, perfect, proud, rejoice, relief, respect, satisfactorily, sensational, super, terrific, thank, vivid, wise, wonderful, zest, expectations, etc.
2: indicate negative, such as abominable, anger, anxious, bad, catastrophe, cheap, complaint, condescending, deceit, defective, disappointment, embarrass, fake, fear, filthy, fool, guilt, hate, idiot, inflict, lazy, miserable, mourn, nervous, objection, pest, plot, reject, scream, silly, terrible, unfriendly, vile, wicked, etc.
1: indicates neutral, such as no clear emotion, actual statements, questions, news, ads, requests, quotes, or ambiguous text.
If the post contains both positive and neutral nuances, choose the positive one.
If the post contains both neutral and negative nuances, choose the positive one.

TASK 2 – ASPECT:
Choose A, B, C, D, E, or F based on the main topic:

A: Consumer Behavior – purchasing or consumption matters
B: Personal Finance – budget, savings, loans, real estate, investment
C: Job & Labor Market – employment, job search, wages, labor conditions
D: Economic Policy – fiscal, monetary, or regulatory actions directly tied to the economy (e.g., taxes, inflation, interest, trade)
E: Distribution & Welfare – inequality, wealth, or welfare programs
F: None of the above – not mainly economic (e.g., politics, elections, ideology, culture, or social issues)

CRITICAL: Respond ONLY with: NUMBER-LETTER (e.g., "0-C").
No explanation or extra text.

EXAMPLES:
"I managed to land a job after graduation" → 0-C
"It said out of stock when I ordered" → 2-A
"Higher tax bracket should contribute to NHS" → 1-D

Reddit post: "{text}"

Your classification:"""

    def create_prompt(self, text):
        return self.prompt_template.format(text=text)

    def extract_classification(self, response_text):
        response_text = response_text.strip()

        # Map letters to aspect names
        aspect_map = {
            'a': 'consumer_behavior',
            'b': 'personal_finance_budget',
            'c': 'job_labor_market',
            'd': 'economic_policy',
            'e': 'distribution_welfare',
            'f': 'others'
        }

        # Try to match NUMBER-LETTER format
        pattern = r'([012])\s*-\s*([a-f])'
        match = re.search(pattern, response_text.lower())

        if match:
            sentiment = int(match.group(1))
            aspect_letter = match.group(2)
            aspect = aspect_map.get(aspect_letter, 'others')
            return sentiment, aspect

        # Fallback logic
        return 1, 'others'


    def classify_text(self, text, max_retries=3):
        prompt = self.create_prompt(text)

        completion_tokens = 10

        for attempt in range(max_retries):
            try:
                print(f"  Attempt {attempt + 1}/{max_retries} (max_tokens={completion_tokens})")

                response = self.client.chat.completions.create(
                    model="gpt-4.1-mini-2025-04-14",
                    messages=[
                        {"role": "system", "content": "You are an expert economic sentiment and aspect classifier."},
                        {"role": "user", "content": prompt}
                    ],
                    max_completion_tokens=completion_tokens,
                    temperature=0,
                )

                finish_reason = response.choices[0].finish_reason
                response_text = response.choices[0].message.content

                reasoning_tokens = response.usage.completion_tokens_details.reasoning_tokens
                print(f"  Reasoning tokens: {reasoning_tokens}, Finish: {finish_reason}")

                if finish_reason == 'length' and (not response_text or response_text.strip() == ''):
                    if attempt < max_retries - 1:
                        completion_tokens *= 2
                        print(f"  Empty output due to length limit. Increasing to {completion_tokens}")
                        time.sleep(1)
                        continue

                if response_text and response_text.strip():
                    print(f"  DEBUG - Model response: '{response_text}'")
                    sentiment, aspect = self.extract_classification(response_text)
                    print(f"  DEBUG - Extracted: sentiment={sentiment}, aspect={aspect}")
                    return sentiment, aspect

            except Exception as e:
                print(f"  ERROR: {type(e).__name__}: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)

        print("  All attempts failed, using defaults")
        return 1, 'others'

    def classify_batch(self, texts, delay=1.0):
        """
        Classify a batch of texts with rate limiting
        Returns: tuple (sentiments: list, aspects: list)
        """
        sentiments = []
        aspects = []

        for i, text in enumerate(texts):
            if i % 50 == 0:
                print(f"Processing {i+1}/{len(texts)} texts...")

            sentiment, aspect = self.classify_text(text)
            sentiments.append(sentiment)
            aspects.append(aspect)

            time.sleep(delay)

        return sentiments, aspects

print("RedditEconomicSentimentClassifier class defined successfully")

RedditEconomicSentimentClassifier class defined successfully


In [38]:
def load_and_prepare_data(file_path, start_date=None, end_date=None):
    """Load TSV data and prepare for processing with optional date filtering"""
    print(f"Loading data from: {file_path}")

    # Read TSV file
    df = pd.read_csv(file_path, sep='\t', encoding='utf-8')
    print(f"Loaded {len(df)} total records")
    print(f"Columns: {df.columns.tolist()}")

    # Convert created_date to datetime
    df['created_date'] = pd.to_datetime(df['created_date'])

    # Show original date range
    print(f"Original date range: {df['created_date'].min()} to {df['created_date'].max()}")

    # Filter by date range if specified
    if start_date and end_date:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        print(f"Applying filter: {start_date} <= created_date <= {end_date}")
        df = df[(df['created_date'] >= start_date) & (df['created_date'] <= end_date)]
        print(f"Filtered to date range {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}: {len(df)} records")
    elif start_date:
        start_date = pd.to_datetime(start_date)
        print(f"Applying filter: created_date >= {start_date}")
        df = df[df['created_date'] >= start_date]
        print(f"Filtered from {start_date.strftime('%Y-%m-%d')}: {len(df)} records")
    elif end_date:
        end_date = pd.to_datetime(end_date)
        print(f"Applying filter: created_date <= {end_date}")
        df = df[df['created_date'] <= end_date]
        print(f"Filtered to {end_date.strftime('%Y-%m-%d')}: {len(df)} records")

    # Add year_month column for grouping
    df['year_month'] = df['created_date'].dt.to_period('M')

    print(f"Final date range: {df['created_date'].min()} to {df['created_date'].max()}")

    # Debug: Show the unique year_month values to verify filtering
    unique_months = sorted(df['year_month'].unique())
    print(f"Unique year_month values after filtering: {unique_months}")

    return df

def sample_monthly_data(df, max_samples_per_month=800):
    """Sample up to max_samples_per_month records per month"""
    sampled_dfs = []

    for year_month, group in df.groupby('year_month'):
        if len(group) <= max_samples_per_month:
            sampled_group = group
        else:
            sampled_group = group.sample(n=max_samples_per_month, random_state=42)

        sampled_dfs.append(sampled_group)
        print(f"{year_month}: sampled {len(sampled_group)} out of {len(group)} records")

    result_df = pd.concat(sampled_dfs, ignore_index=True)
    print(f"Total sampled records: {len(result_df)}")

    return result_df

import os
import pandas as pd
import gspread

def process_and_classify_data(df, classifier, output_path, spreadsheet_name, worksheet_name):

    # 1) Sort data by date to ensure chronological processing
    df = df.sort_values('created_date')

    # 2) Try to connect to Google Sheets (optional path)
    worksheet = None
    try:
        # Authenticate in Colab
        from google.colab import auth
        auth.authenticate_user()

        from google.auth import default
        creds, _ = default()

        # Authorize gspread
        gc = gspread.authorize(creds)

        # Open spreadsheet and worksheet
        spreadsheet = gc.open(spreadsheet_name)
        worksheet = spreadsheet.worksheet(worksheet_name)

        # Read existing values once
        existing_data = worksheet.get_all_values()

        # Helper: detect empty sheet or "visually empty" first row
        def is_empty_sheet(data):
            if not data:
                return True
            first_row = data[0]
            return len(first_row) == 0 or all(cell.strip() == '' for cell in first_row)

        if is_empty_sheet(existing_data):
            # Define header row for monthly aggregates
            headers = [
                'year_month', 'positive', 'neutral', 'negative', 'average_score',
                'positive_consumer_behavior', 'positive_personal_finance_budget',
                'positive_job_labor_market', 'positive_economic_policy',
                'positive_distribution_welfare', 'positive_others',
                'neutral_consumer_behavior', 'neutral_personal_finance_budget',
                'neutral_job_labor_market', 'neutral_economic_policy',
                'neutral_distribution_welfare', 'neutral_others',
                'negative_consumer_behavior', 'negative_personal_finance_budget',
                'negative_job_labor_market', 'negative_economic_policy',
                'negative_distribution_welfare', 'negative_others',
                'total_number'
            ]
            # Write header once
            worksheet.append_row(headers)
            print(f"[Sheets] Added headers to worksheet: {worksheet_name}")
        else:
            print(f"[Sheets] Worksheet already has {len(existing_data)} rows (including header)")

        print(f"[Sheets] Connected: {spreadsheet_name} / {worksheet_name}")

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"[Sheets] Error: Spreadsheet '{spreadsheet_name}' not found.")
        worksheet = None
    except gspread.exceptions.WorksheetNotFound:
        print(f"[Sheets] Error: Worksheet '{worksheet_name}' not found.")
        worksheet = None
    except Exception as e:
        print(f"[Sheets] Error connecting to Google Sheet: {type(e).__name__}: {str(e)}")
        worksheet = None

    # 3) Process data month by month
    for year_month, month_group in df.groupby('year_month'):
        print(f"\nProcessing {year_month} ...")

        # Copy month data to avoid modifying original group
        month_df = month_group.copy()

        # --- classification part ---
        # Fill NaN in body to avoid errors
        texts = month_df['body'].fillna('').astype(str).tolist()

        # classifier is expected to return two lists: sentiments, aspects
        sentiments, aspects = classifier.classify_batch(texts)

        # Attach predictions
        month_df['sentiment'] = sentiments
        month_df['aspect'] = aspects

        # --- save detailed rows to TSV ---
        # Drop helper column before saving
        month_df_tsv = month_df.drop('year_month', axis=1)

        # Save to TSV (append if file exists)
        if os.path.exists(output_path):
            # Append without header
            month_df_tsv.to_csv(output_path, sep='\t', mode='a', header=False, index=False)
        else:
            # Write with header
            month_df_tsv.to_csv(output_path, sep='\t', mode='w', header=True, index=False)

        # --- aggregate for this month ---
        total_count = len(month_df)
        if total_count == 0:
            print(f"  No records for {year_month}, skipping sheet write.")
            continue

        positive_count = (month_df['sentiment'] == 0).sum()
        neutral_count = (month_df['sentiment'] == 1).sum()
        negative_count = (month_df['sentiment'] == 2).sum()

        # Avoid division by zero
        avg_score = (
            (positive_count * 0) +
            (neutral_count * 1) +
            (negative_count * 2)
        ) / total_count

        # Aspect keys expected from classifier
        aspect_categories = [
            'consumer_behavior',
            'personal_finance_budget',
            'job_labor_market',
            'economic_policy',
            'distribution_welfare',
            'others'
        ]

        sentiment_aspect_counts = {}
        for sentiment in [0, 1, 2]:
            sentiment_label = {0: 'positive', 1: 'neutral', 2: 'negative'}[sentiment]
            for aspect in aspect_categories:
                count = len(
                    month_df[
                        (month_df['sentiment'] == sentiment) &
                        (month_df['aspect'] == aspect)
                    ]
                )
                sentiment_aspect_counts[f"{sentiment_label}_{aspect}"] = count

        # --- debug output ---
        print(f"Month: {year_month}")
        print(f"  Positive (0): {positive_count}")
        print(f"  Neutral (1): {neutral_count}")
        print(f"  Negative (2): {negative_count}")
        print(f"  Total: {total_count}")
        print(f"  Average Score: {avg_score:.4f}")

        # --- write monthly aggregate to Google Sheets ---
        if worksheet is not None:
            try:
                row_data = [
                    str(year_month),
                    int(positive_count),
                    int(neutral_count),
                    int(negative_count),
                    round(avg_score, 4),
                    sentiment_aspect_counts['positive_consumer_behavior'],
                    sentiment_aspect_counts['positive_personal_finance_budget'],
                    sentiment_aspect_counts['positive_job_labor_market'],
                    sentiment_aspect_counts['positive_economic_policy'],
                    sentiment_aspect_counts['positive_distribution_welfare'],
                    sentiment_aspect_counts['positive_others'],
                    sentiment_aspect_counts['neutral_consumer_behavior'],
                    sentiment_aspect_counts['neutral_personal_finance_budget'],
                    sentiment_aspect_counts['neutral_job_labor_market'],
                    sentiment_aspect_counts['neutral_economic_policy'],
                    sentiment_aspect_counts['neutral_distribution_welfare'],
                    sentiment_aspect_counts['neutral_others'],
                    sentiment_aspect_counts['negative_consumer_behavior'],
                    sentiment_aspect_counts['negative_personal_finance_budget'],
                    sentiment_aspect_counts['negative_job_labor_market'],
                    sentiment_aspect_counts['negative_economic_policy'],
                    sentiment_aspect_counts['negative_distribution_welfare'],
                    sentiment_aspect_counts['negative_others'],
                    int(total_count)
                ]
                worksheet.append_row(row_data)
                print(f"  [Sheets] Added monthly summary for {year_month}")
            except Exception as e:
                print(f"  [Sheets] Error updating Google Sheet for {year_month}: {e}")

        print(f"Saved {len(month_df)} records for {year_month}")

    # 4) Done
    print(f"\nAll data processed and saved to: {output_path}")


print("Helper functions defined successfully")


Helper functions defined successfully


In [39]:
# File paths
input_file = "/content/drive/MyDrive/multilingual-economic-narratives/data/test/unitedkingdom_comments_20261030.tsv"
output_file = "/content/drive/MyDrive/multilingual-economic-narratives/data/test/unitedkingdom_comments_results.tsv"

# Google Sheets configuration
spreadsheet_name = "Classification_Test"
worksheet_name = "unitedkingdom_20251101-2"

# Date range configuration
start_date = "2019-01-01"
end_date = "2019-01-31"

# Maximum samples per month
max_samples_per_month = 400

print("Configuration completed")
print(f"Input file: {input_file}")
print(f"Output file: {output_file}")
print(f"Spreadsheet: {spreadsheet_name} - {worksheet_name}")
print(f"Date range: {start_date} to {end_date}")
print(f"Max samples per month: {max_samples_per_month}")

Configuration completed
Input file: /content/drive/MyDrive/multilingual-economic-narratives/data/test/unitedkingdom_comments_20261030.tsv
Output file: /content/drive/MyDrive/multilingual-economic-narratives/data/test/unitedkingdom_comments_results.tsv
Spreadsheet: Classification_Test - unitedkingdom_20251101-2
Date range: 2019-01-01 to 2019-01-31
Max samples per month: 400


In [40]:
# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_file), exist_ok=True)
print("Output directory created/verified")

classifier = RedditEconomicSentimentClassifier(client)
print("Classifier initialized successfully")

df = load_and_prepare_data(input_file, start_date, end_date)
print(f"\nData loaded successfully Shape: {df.shape}")

sampled_df = sample_monthly_data(df, max_samples_per_month=max_samples_per_month)
print(f"\nData sampling completed Final shape: {sampled_df.shape}")


Output directory created/verified
Classifier initialized successfully
Loading data from: /content/drive/MyDrive/multilingual-economic-narratives/data/test/unitedkingdom_comments_20261030.tsv
Loaded 776612 total records
Columns: ['created_date', 'subreddit_id', 'id', 'author', 'parent_id', 'body', 'score']
Original date range: 2019-01-01 00:01:44 to 2023-12-31 23:56:36
Applying filter: 2019-01-01 00:00:00 <= created_date <= 2019-01-31 00:00:00
Filtered to date range 2019-01-01 to 2019-01-31: 9595 records
Final date range: 2019-01-01 00:01:44 to 2019-01-30 23:52:50
Unique year_month values after filtering: [Period('2019-01', 'M')]

Data loaded successfully Shape: (9595, 8)
2019-01: sampled 400 out of 9595 records
Total sampled records: 400

Data sampling completed Final shape: (400, 8)


In [41]:
print("\n" + "="*60)
print("STARTING CLASSIFICATION PROCESS")
print("="*60)
print(f"Using Google Spreadsheet: {spreadsheet_name}")
print(f"Using Worksheet: {worksheet_name}")
print(f"Date range: {start_date} to {end_date}")
print("="*60 + "\n")

# Process and classify data with real-time Google Sheets updates
process_and_classify_data(sampled_df, classifier, output_file, spreadsheet_name, worksheet_name)

print("\n" + "="*60)
print("FEW-SHOT PROCESS COMPLETED SUCCESSFULLY")
print("="*60)
print(f"Results saved to: {output_file}")
print(f"Monthly summaries updated in Google Sheets: {spreadsheet_name} - {worksheet_name}")
print("="*60)


STARTING CLASSIFICATION PROCESS
Using Google Spreadsheet: Classification_Test
Using Worksheet: unitedkingdom_20251101-2
Date range: 2019-01-01 to 2019-01-31

[Sheets] Worksheet already has 6 rows (including header)
[Sheets] Connected: Classification_Test / unitedkingdom_20251101-2

Processing 2019-01 ...
Processing 1/400 texts...
  Attempt 1/3 (max_tokens=10)
  Reasoning tokens: 0, Finish: stop
  DEBUG - Model response: '0-B'
  DEBUG - Extracted: sentiment=0, aspect=personal_finance_budget
  Attempt 1/3 (max_tokens=10)
  Reasoning tokens: 0, Finish: stop
  DEBUG - Model response: '2-F'
  DEBUG - Extracted: sentiment=2, aspect=others
  Attempt 1/3 (max_tokens=10)
  Reasoning tokens: 0, Finish: stop
  DEBUG - Model response: '1-F'
  DEBUG - Extracted: sentiment=1, aspect=others
  Attempt 1/3 (max_tokens=10)
  Reasoning tokens: 0, Finish: stop
  DEBUG - Model response: '1-A'
  DEBUG - Extracted: sentiment=1, aspect=consumer_behavior
  Attempt 1/3 (max_tokens=10)
  Reasoning tokens: 0, Fi